In [ ]:
import holodeck as holo
from holodeck import single_sources, utils, plot, detstats
from holodeck.constants import YR

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py



# Anisotropy Functions

In [ ]:
NSIDE = 24
LMAX = 8

def healpix_map(hc_ss, hc_bg, nside=NSIDE):
    """ Build mollview array of strains for a healpix map
    
    Parameters
    ----------
    hc_ss : (F,R,L) NDarray
        Characteristic strain of single sources.
    hc_bg : (F,R) NDarray
        Characteristic strain of the background.
    nside : integer
        number of sides for healpix map.

    Returns
    -------
    moll_hc : (NPIX,) 1Darray
        Array of strain at every pixel for a mollview healpix map.
    
    NOTE: Could speed up the for-loops, but it's ok for now.
    """

    npix = hp.nside2npix(nside)
    nfreqs = len(hc_ss)
    nreals = len(hc_ss[0])
    nloudest = len(hc_ss[0,0])

    # spread background evenly across pixels in moll_hc
    moll_hc = np.ones((nfreqs,nreals,npix)) * hc_bg[:,:,np.newaxis]/np.sqrt(npix) # (frequency, realization, pixel)

    # choose random pixels to place the single sources
    pix_ss = np.random.randint(0, npix-1, size=nfreqs*nreals*nloudest).reshape(nfreqs, nreals, nloudest)
    for ff in range(nfreqs):
        for rr in range(nreals):
            for ll in range(nloudest):
                moll_hc[ff,rr,pix_ss[ff,rr,ll]] = np.sqrt(moll_hc[ff,rr,pix_ss[ff,rr,ll]]**2
                                                          + hc_ss[ff,rr,ll]**2)
                
    return moll_hc

def sph_harm_from_map(moll_hc, lmax=LMAX):
    """ Calculate spherical harmonics from strains at every pixel of 
    a healpix mollview map.
    
    Parameters
    ----------
    moll_hc : (F,R,NPIX,) 1Darray
        Characteristic strain of each pixel of a healpix map.
    lmax : int
        Highest harmonic to calculate.

    Returns
    -------
    Cl : (F,R,lmax+1) NDarray
        Spherical harmonic coefficients 
        
    """
    nfreqs = len(moll_hc)
    nreals = len(moll_hc[0])

    Cl = np.zeros((nfreqs, nreals, lmax+1))
    for ff in range(nfreqs):
        for rr in range(nreals):
            Cl[ff,rr,:] = hp.anafast(moll_hc[ff,rr], lmax=lmax)

    return Cl

def sph_harm_from_hc(hc_ss, hc_bg, nside = NSIDE, lmax = LMAX):
    """ Calculate spherical harmonics and strain at every pixel
    of a healpix mollview map from single source and background char strains.

    Parameters
    ----------
    hc_ss : (F,R,L) NDarray
        Characteristic strain of single sources.
    hc_bg : (F,R) NDarray
        Characteristic strain of the background.
    nside : integer
        number of sides for healpix map.

    Returns
    -------
    moll_hc : (F,R,NPIX,) 2Darray
        Array of strain at every pixel for a mollview healpix map.
    Cl : (F,R,lmax+1) NDarray
        Spherical harmonic coefficients 
    
    """
    moll_hc = healpix_map(hc_ss, hc_bg, nside)
    Cl = sph_harm_from_map(moll_hc, lmax)

    return moll_hc, Cl



##########################################################################################
################################### PLOTTING FUNCTIONS ###################################
##########################################################################################

def plot_Cl(Cl, lvals, fobs_nHz, ff_labels=(0, 10, 20, 30, 39)):
    """ Plot C_l vs l for all frequencies of a single realization.
    
    """
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')
    colors=cm.rainbow(np.linspace(0,1,len(Cl)))

    for ff in range(len(Cl)):
        if ff in ff_labels:
            label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
        else: label = None
        ax.plot(lvals[:], Cl[ff,:], c=colors[ff], alpha=0.5, label=label,
                marker='o')
        
    ax.legend()
    return fig

def plot_ClC0(Cl, lvals, fobs_nHz, ff_labels=(0, 10, 20, 30, 39)):
    """ Plot C_l/C_0 for all frequencies of a single realization.
    
    """
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')
    colors=cm.rainbow(np.linspace(0,1,len(Cl)))

    for ff in range(len(Cl)):
        if ff in ff_labels:
            label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
        else: label = None
        ax.plot(lvals[:], Cl[ff,:]/Cl[ff,0,np.newaxis], c=colors[ff], alpha=0.5, label=label,
                marker='o')
        
    ax.legend()
    return fig

def plot_llp1Cl(Cl, lvals, fobs_nHz, ff_labels=(0, 10, 20, 30, 39)):
    """ Plot l(l+1)C_l vs l for all frequencies of a single realization.
    
    """
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$\ell(\ell+1)C_\ell$', xscale='linear')
    colors=cm.rainbow(np.linspace(0,1,len(Cl)))

    xx = lvals[1:]
    yy = (lvals*(lvals+1))[np.newaxis,1:]*Cl[:,1:]
    for ff in range(len(Cl)):
        if ff in ff_labels:
            label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
        else: label = None
        ax.plot(xx, yy[ff], c=colors[ff], alpha=0.5, label=label,
                marker='o')
        
    ax.legend()
    return fig

def plot_Clg0(Cl, fobs):
    """ Plot C_l>0/C_0 vs f for many realizations.
    
    """

    fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$', xscale='linear')
    xx = fobs*YR
    print(xx.shape)
    print(np.sum(Cl[:,:,1:], axis=2).shape)
    print(Cl[:,:,0,np.newaxis].shape)
    yy = np.sum(Cl[:,:,1:], axis=2)/Cl[:,:,0]
    print(yy.shape)
    for rr in range(len(Cl[0])):
        ax.plot(xx, yy[:,rr], alpha=0.5, marker='o')

    ax.plot(xx,np.median(yy, axis=1), color='k')
        
    ax.legend()
    return fig

def map_from_sph_harm(Cl, nside=NSIDE, lmax=LMAX):
    nfreqs = len(Cl)
    nreals = len(Cl[0])
    npix = hp.nside2npix(nside)
    moll_sh = np.zeros((nfreqs,nreals,npix))
    for ff in range(nfreqs):
        for rr in range(nreals):
            moll_sh[ff,rr,:] = hp.synfast(Cl[ff,rr], nside, lmax=lmax)
    return moll_sh

# Read in Library

ss16 has 10 loudest, definitely will replace this with a better library

In [ ]:
sspath = '/Users/emigardiner/GWs/holodeck/output/2023-05-16-mbp-ss19_uniform05A_n1000_r50_d20_f30_l2000_p0/'
# hdfname = sspath+'ss_lib.hdf5'
# ssfile = h5py.File(hdfname, 'r')
# print(list(ssfile.keys()))
# hc_ss = ssfile['hc_ss'][...]
# hc_bg = ssfile['hc_bg'][...]
# fobs = ssfile['fobs'][:]
# dfobs = ssfile['dfobs'][:]
# ssfile.close()

# shape = hc_ss.shape
# nsamps, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
# print('N,F,R,L =', nsamps, nfreqs, nreals, nloudest)


# Find Best Samples

In [ ]:
# hc_ref15_10yr = 11.2*10**-15 
# nsort, fidx, hc_tt, hc_ref15 = detstats.rank_samples(hc_ss, hc_bg, fobs, hc_ref=hc_ref15_10yr, ret_all=True)
# nbest = 100
# print(hc_ref15)


### Plot amplitudes sorted

In [ ]:
# # amplitudes from sorting
# fig, ax = plot.figax(figsize=(7,5), xlabel='Sample', ylabel='$h_{c,\mathrm{tot}}(f=1/\mathrm{yr})$',
#                      xscale='linear')

# ax.scatter(nsort[:nbest], hc_tt[nsort][:nbest], s=20, marker='o', c=np.arange(len(nsort))[:nbest], 
#            cmap='jet_r', label='%d best samples' % nbest)
# ax.scatter(nsort[nbest:], hc_tt[nsort][nbest:], s=15, marker='x', c='darkblue', 
#            label='excluded samples')
# ax.axhline(hc_ref15, label='15 yr ref: %.2e' % (hc_ref15), linestyle='dashed', color='tab:red', alpha=0.5)
# # plt.scatter(np.arange(nsamps), hc_tt[nsort], s=3, label= r'15 yr ref: $A=$' + '%.2f' % amp_ref15)

# ax.set_yscale('log')
# ax.set_xlabel('Samples')
# ax.set_ylabel('$h_c(f=%.2f\mathrm{yr}^{-1})$' % (fobs[fidx]*YR))
# ax.legend()

In [ ]:
# # amplitudes from sorting
# fig, ax = plot.figax(figsize=(7,5), xlabel='Sample', ylabel='$h_{c,\mathrm{tot}}(f=1/\mathrm{yr})$',
#                      xscale='linear')
# ax.scatter(nsort, hc_tt[nsort], s=20, marker='o', c=np.arange(len(nsort)), 
#            cmap='jet_r')
# ax.axhline(hc_ref15, label='15 yr ref: %.2e' % (hc_ref15), linestyle='dashed', color='tab:red', alpha=0.5)
# # plt.scatter(np.arange(nsamps), hc_tt[nsort], s=3, label= r'15 yr ref: $A=$' + '%.2f' % amp_ref15)

# ax.set_yscale('log')
# ax.set_xlabel('Samples')
# ax.set_ylabel('$h_c(f=%.2f\mathrm{yr}^{-1})$' % (fobs[fidx]*YR))
# ax.legend()

### Plot strains from best samples

In [ ]:
# print(nsort[:5])
# xx = fobs*YR
# yy = hc_ref15_10yr * (xx/.1)**(-2/3)
# for nn in nsort[:5]:
#     print('nn=%d' % nn)
#     fig = plot.plot_bg_ss(fobs, hc_bg[nn], hc_ss[nn,:,:,:100]) # just 100 loudest, cant see more anyway
#     ax = fig.axes[0]
#     ax.plot(xx,yy,linestyle='dashdot', color='grey', alpha=1, 
#             label= '$(%.2e) [f/(.1 \mathrm{yr^-1})]^{-2/3}  $' % hc_ref15_10yr)
#     ax.axhline(hc_ref15, label='15 yr ref: $h_c(%.2f/\mathrm{yr}) = %.2e$' % (xx[fidx], hc_ref15), linestyle='dashed', color='tab:red', alpha=0.5)
#     ax.axvline(xx[fidx], color='tab:blue', alpha=0.5, linestyle='dashed', label='$f=%.2f\mathrm{yr}$' % xx[fidx])
#     ax.set_title('Sample %d' % nn)
#     ax.legend(loc='lower left')

# Load/Calculate pixel strains and harmonic coefficients

In [ ]:
# nside = NSIDE
# npix = hp.nside2npix(nside)
# lmax = LMAX
# Cl_best = np.zeros((nbest, nfreqs, nreals, lmax+1 ))
# moll_hc_best = np.zeros((nbest, nfreqs, nreals, npix))
# for nn in range(nbest):
#     print('on nn=%d out of nbest=%d' % (nn,nbest))
#     moll_hc_best[nn,...], Cl_best[nn,...] = sph_harm_from_hc(hc_ss[nsort[nn]], 
#                                             hc_bg[nsort[nn]],
#                                             nside=nside, lmax=lmax)

In [ ]:
# np.savez(sspath+'sph_harmonics.npz', nsort=nsort, fidx=fidx, hc_tt=hc_tt, hc_ref15=hc_ref15, ss_shape=shape,
#          moll_hc_best=moll_hc_best, Cl_best=Cl_best, nside=nside, lmax=lmax, fobs=fobs)

sph_harm_file = np.load(sspath+'sph_harmonics.npz')

# load ss info
shape = sph_harm_file['ss_shape']
nsamps, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
fobs = sph_harm_file['fobs']

# load ranking info
nsort = sph_harm_file['nsort']
fidx = sph_harm_file['fidx']
hc_tt = sph_harm_file['hc_tt']
hc_ref15 = sph_harm_file['hc_ref15']

# load harmonics info
nside = sph_harm_file['nside']
lmax  = sph_harm_file['lmax']
moll_hc_best = sph_harm_file['moll_hc_best']
Cl_best = sph_harm_file['Cl_best']
nbest = len(moll_hc_best)

sph_harm_file.close()

### Plot Results

In [ ]:
lvals = np.arange(lmax+1)
fobs_nHz = fobs*10**9
fobs_yrs = fobs*YR
ff_labels = (0,9,19,29,39)
nshow = 10

#### Plot_Cl_medians

In [ ]:
def plot_Cl_medians(lvals, Cl_best, lmax):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')

    xx = lvals # (l)
    yy = Cl_best # (B,F,R,l)
    yy = np.median(yy, axis=2) # (B,F,l)

    colors = cm.rainbow(np.linspace(1, 0, nfreqs+1))
    for ff in (np.arange(1,nfreqs,8)):
        label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
        ax.plot(xx, np.median(yy[:,ff,:], axis=0), color=colors[ff], label=label,
                linewidth=3)
        for pp in [50,]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[:,ff,:], percs, axis=0), alpha=0.25, color=colors[ff])
        for bb in range(nshow):
            ax.plot(xx, yy[bb,ff,:], color=colors[ff], linestyle=':', alpha=0.75)

    ax.legend()
    ax.set_title('50%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
                % (nbest, lmax))
    return fig

fig = plot_Cl_medians(lvals, Cl_best, lmax)

In [ ]:
fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')

xx = lvals # (l)
yy = Cl_best # (B,F,R,l)
yy = np.median(yy, axis=2) # (B,F,l)

# choose a frequency 
ff = fidx # f=1/yr
label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
yy = yy[:,ff,:] # (B,l)

colors = cm.rainbow(np.linspace(1, 0, 31))
ax.plot(xx, np.median(yy, axis=0), color=colors[ff], label=label, lw='3')
for pp in [50, 98]:
    percs = pp/2
    percs = [50-percs, 50+percs]
    ax.fill_between(xx, *np.percentile(yy, percs, axis=0), alpha=0.25, color=colors[ff])

colors = cm.rainbow(np.linspace(0,1,nshow))
for bb in range(nshow):
    ax.plot(xx, yy[bb,:], color=colors[bb], linestyle=':', alpha=0.75)
ax.legend()
ax.set_title('50%% and 98%% Confidence Intervals of the %d Best Samples, \nusing median among realizations, lmax=%d' % (nbest, lmax))

#### plot_ClC0_medians

In [ ]:
def plot_ClC0_medians(lvals, Cl_best, lmax):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')

    xx = lvals[1:] # (l)
    yy = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)
    yy = np.median(yy, axis=2) # (F,R,l)

    colors = cm.rainbow(np.linspace(1, 0, nfreqs+1))
    for ff in (np.arange(0,nfreqs,8)):
        label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
        ax.plot(xx, np.median(yy[:,ff,:], axis=0), color=colors[ff], label=label,
                linewidth=4)
        for pp in [50,]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[:,ff,:], percs, axis=0), alpha=0.25, color=colors[ff],
                            linestyle='solid', linewidth=3)
        for bb in range(nshow):
            ax.plot(xx, yy[bb,ff,:], color=colors[ff], linestyle=':', alpha=0.75)

    fig.legend(ncol=3, bbox_to_anchor=(-.01,-.27), 
            bbox_transform=ax.transAxes, loc='lower left')
    ax.set_title('50%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
                % (nbest, lmax))
    fig.tight_layout()
    return fig

fig = plot_ClC0_medians(lvals, Cl_best, lmax)

In [ ]:
fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')

xx = lvals[1:] # (l)
yy = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)
yy = np.median(yy, axis=2) # (B,F,l)

# choose a frequency 
ff = fidx # f=1/yr
label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
yy = yy[:,ff,:] # (B,l)

colors = cm.rainbow(np.linspace(1, 0, 31))
ax.plot(xx, np.median(yy, axis=0), color=colors[ff], label=label, lw=4)
for pp in [50, 98]:
    percs = pp/2
    percs = [50-percs, 50+percs]
    ax.fill_between(xx, *np.percentile(yy, percs, axis=0), alpha=0.25, color=colors[ff])

colors = cm.rainbow(np.linspace(0,1,nbest))
for bb in range(nbest):
    ax.plot(xx, yy[bb,:], color=colors[bb], linestyle=':', alpha=0.75)
ax.legend()
ax.set_title('50%% and 98%% Confidence Intervals of the %d Best Samples, \nusing median among realizations' % (nbest))

Find medium, small, and large anistotropy runs among these 100 samples

#### plot_Clg0_medians

In [ ]:
def plot_Clg0_medians(Cl_best, lmax):
    fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')

    xx = fobs_yrs
    yy = np.sum(Cl_best[:,:,:,1:], axis=-1)/Cl_best[:,:,:,0,] # (B,F,R)
    yy = np.median(yy, axis=-1) # (B,F) median over realizations

    ax.plot(xx, np.median(yy, axis=0), color='k')
    for pp in [50, 98]:
        percs = pp/2
        percs = [50-percs, 50+percs]
        ax.fill_between(xx, *np.percentile(yy, percs, axis=0), alpha=0.2, color='k')

    colors = cm.rainbow(np.linspace(1, 0, nshow))
    for bb in range(nshow):
        ax.plot(xx, yy[bb,:], color=colors[bb], linestyle=':', alpha=0.75,
                linewidth=2)
    ax.legend()
    ax.set_title('50%% and 98%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
                % (nbest, lmax))
    return fig
fig = plot_Clg0_medians(Cl_best, lmax)

In [ ]:
fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')

xx = fobs_yrs
yy = np.sum(Cl_best[:,:,:,1:], axis=-1)/Cl_best[:,:,:,0,] # (B,F,R)

colors = cm.rainbow(np.linspace(1, 0, 5))
rr=0
for bb in range(5):
    # label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
    label='p = %d' % nsort[bb]
    ax.plot(xx, np.median(yy[bb,:,:], axis=-1), color=colors[bb], label=label)
    for pp in [50, 98]:
        percs = pp/2
        percs = [50-percs, 50+percs]
        ax.fill_between(xx, *np.percentile(yy[bb,:,:], percs, axis=-1), alpha=0.25, color=colors[bb])
ax.legend()
ax.set_title('50%% and 98%% confidence intervals from %d realizations of the %d best samples' % (nreals,5))

# Look at 1 sample, more closely

In [ ]:
bb = 0 # best sample
nn = nsort[bb]
rr = 0
print(Cl_best.shape)
print(nn)

In [ ]:
fig1 = plot_Cl(Cl_best[bb,:,rr,:], lvals, fobs_nHz, ff_labels=(0,9,19,29,39))
fig1.axes[0].set_title('Sample %d, Realization %d' % (nn,rr))

fig2 = plot_ClC0(Cl_best[bb,:,rr,:], lvals, fobs_nHz, ff_labels=(0,9,19,29,39))
fig2.axes[0].set_title('Sample %d, Realization %d' % (nn,rr))

fig3 = plot_llp1Cl(Cl_best[bb,:,rr,:], lvals, fobs_nHz, ff_labels=(0,9,19,29,39))
fig3.axes[0].set_title('Sample %d, Realization %d' % (nn,rr))

fig4 = plot_Clg0(Cl_best[bb,:,:,:], fobs)

## Look at 3 cases, highest, lowest, and middle anisotropy

In [ ]:
ClC0_ratio = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)
ClC0_mean = np.mean(ClC0_ratio, axis = (1,2,3)) # just one value for each 'best' sample, to quantify total anisotropy
fig, ax = plot.figax(xscale='linear', xlabel='best sample', ylabel='mean $C_\ell/C_0$')
ax.scatter(np.arange(nbest), ClC0_mean)
ax.set_title('mean $C_\ell/C_0$ of all frequencies, realizations, and $\ell>0$')

Find min, middle, max anisotropy samples

In [ ]:
def argmedian(arr, side='both'):
    if len(arr) % 2 == 1:
        return np.where(arr == np.median(arr))[0][0]
    else:
        ll,rr = len(arr) // 2 - 1, len(arr) // 2
        left = np.partition(arr, ll)[ll]
        right = np.partition(arr, rr)[rr]
        if(side=='both'):
            return [np.where(arr == left)[0][0], np.where(arr == right)[0][0]]
        elif(side=='left'):
            return np.where(arr == left)[0][0]
        elif(side=='right'):
            return np.where(arr==right)[0][0]
        else:
            raise Exception("'side' must be one of 'both', 'left', or 'right'.")

bb_min = np.argmin(ClC0_mean)
bb_max = np.argmax(ClC0_mean)
bb_med = argmedian(ClC0_mean, side='left')
nn_min, nn_max, nn_med = nsort[bb_min], nsort[bb_max], nsort[bb_med]
print('Min: Sample %d, Median: Sample %d, Max: Sample %d' % (nn_min, nn_med, nn_max))

## Plot_Cl_reals

In [ ]:
def plot_Cl_reals(lvals, Cl, lmax, bb):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')
    
    xx = lvals # (l)
    yy = Cl # (F,R,l)

    ff_arr = np.arange(1, nfreqs, 8)
    
    draw_freq_reals(ax, xx, yy, ff_arr)
    ax.legend()
    ax.set_title('Sample %d: 50%% and 98%% confidence intervals of the %d realizations, lmax=%d'
                % (nsort[bb], nreals, lmax))
    return fig

def draw_freq_reals(ax, xx, yy, ff_arr, nshow=10, civals = [50, 98], rlabel='',
                ):
    """ draw realization for a single sample at several frequencies, 
    
    Parameters
    ----------
    ax
    xx : (l) 1Darray
    yy : (F,R,l) NDarray
    nshow : integer
        Number of realizations to plot
    """
    
    colors = cm.rainbow(np.linspace(1, 0, len(yy)+1))
    for ff in ff_arr:
        flabel= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
        draw_reals(ax, xx, yy[ff], nshow=nshow, civals = civals, color=colors[ff], flabel=flabel)

def draw_reals(ax, xx, yy, nshow=10, civals = [50, 98], rlabel='', flabel='', color='k'):
    """ draw realization for a single sample at several frequencies, 
    
    Parameters
    ----------
    ax
    xx : (l) 1Darray
    yy : (R,l) NDarray
    nshow : integer
        Number of realizations to plot
    """
    label= (rlabel+flabel)
    ax.plot(xx, np.median(yy[:,:], axis=0), color=color, label=label,
            linewidth=3, alpha=0.75)
    for pp in civals:
        percs = pp/2
        percs = [50-percs, 50+percs]
        ax.fill_between(xx, *np.percentile(yy[:,:], percs, axis=0), alpha=0.25, color=color)
    for rr in range(nshow):
        ax.plot(xx, yy[rr,:], color=color, linestyle=':', alpha=0.75)

    

fig = plot_Cl_reals(lvals, Cl_best[0], lmax, bb=0)
fig = plot_Cl_reals(lvals, Cl_best[bb_med], lmax, bb=bb_med)

## Plot_ClC0_reals

In [ ]:
def plot_ClC0_reals(lvals, Cl, lmax, bb):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')
    
    xx = lvals[1:] # (l)
    yy = Cl # (F,R,l)
    yy = yy[:,:,1:]/yy[:,:,0,np.newaxis]

    ff_arr = np.arange(1, nfreqs, 8)
    
    draw_freq_reals(ax, xx, yy, ff_arr, civals=[50,])
    ax.legend()
    ax.set_title('Sample %d: 50%% confidence intervals of the %d realizations, lmax=%d'
                % (nsort[bb], nreals, lmax))
    return fig

    

fig = plot_ClC0_reals(lvals, Cl_best[0], lmax, bb=0)
fig = plot_ClC0_reals(lvals, Cl_best[bb_med], lmax, bb=bb_med)

## Plot Clg0_reals

In [ ]:
def plot_Clg0_reals(Cl, lmax, bb):
    fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')

    xx = fobs_yrs
    yy = np.sum(Cl[:,:,1:], axis=-1)/Cl[:,:,0,] # (F,R)

        # label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
    ax.plot(xx, np.median(yy[:,:], axis=1), color='k')
    for pp in [50, 98]:
        percs = pp/2
        percs = [50-percs, 50+percs]
        ax.fill_between(xx, *np.percentile(yy[:,:], percs, axis=1), alpha=0.15, color='k')

    for rr in range(20):
        ax.plot(xx, yy[:,rr], linestyle=':', alpha=0.75)
    ax.legend()
    ax.set_title('Sample %d: 50%% and 98%% confidence intervals from %d realizations, lmax=%d' % (nsort[bb], nreals, lmax)) 
    return fig

fig = plot_Clg0_reals(Cl_best[0], lmax, bb=0)
fig = plot_Clg0_reals(Cl_best[bb_med], lmax, bb=bb_med)

## Plot Cl for min, med, max anisotropy

Cl_min_max_med using overall mean selection

In [ ]:
def Cl_min_max_med(lvals, Cl_best, bb_min, bb_med, bb_max, title='', ff=fidx):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')

    xx = lvals # (l)
    y1 = Cl_best[bb_min] # (F,R,l)
    y2 = Cl_best[bb_med] # (F,R,l)
    y3 = Cl_best[bb_max] # (F,R,l)
    y4 = Cl_best[0] # (F,R,l)
    print(Cl_best.shape)
    print(y2.shape)

    draw_reals(ax, xx, y1[ff], rlabel='min (sample %d), ' % nn_min, flabel = ('%.2f $yr^{-1}$' % fobs_yrs[ff]),
            nshow=nshow, civals = [50,], color='tab:blue')
    draw_reals(ax, xx, y2[ff], rlabel='med (sample %d), ' % nn_med, flabel = ('%.2f $yr^{-1}$' % fobs_yrs[ff]),
            nshow=nshow, civals = [50,], color='tab:orange')
    draw_reals(ax, xx, y3[ff], rlabel='max (sample %d), ' % nn_max, flabel = ('%.2f $yr^{-1}$' % fobs_yrs[ff]),
            nshow=nshow, civals = [50,], color='tab:green')
    draw_reals(ax, xx, y4[ff], rlabel='best (sample %d), ' % nsort[0], flabel = ('%.2f $yr^{-1}$' % fobs_yrs[ff]),
            nshow=nshow, civals = [50,], color='tab:grey')
    ax.legend()
    ax.set_title(title)
    return fig

title = ('50%% confidence intervals of %d realizations, lmax=%d\nSelected by mean $C_l/C_0$ over all freqs'
                % (nreals, lmax))
fig = Cl_min_max_med(lvals, Cl_best, bb_min, bb_med, bb_max, title=title, ff=ff)


## Plot Cl/C0 for min, med, max

In [ ]:
def ClC0_min_max_med(lvals, Cl_best, bb_min, bb_med, bb_max, title='', ff=fidx):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')

    xx = lvals[1:] # (l)
    y1 = Cl_best[bb_min] # (F,R,l)
    y2 = Cl_best[bb_med] # (F,R,l)
    y3 = Cl_best[bb_max] # (F,R,l)
    y4 = Cl_best[nsort[0]] # (F,R,l)


    y1 = y1[:,:,1:]/y1[:,:,0,np.newaxis] # (F,R,l)
    y2 = y2[:,:,1:]/y2[:,:,0,np.newaxis]
    y3 = y3[:,:,1:]/y3[:,:,0,np.newaxis]
    y4 = y4[:,:,1:]/y4[:,:,0,np.newaxis]

    draw_reals(ax, xx, y1[ff], rlabel='min (sample %d), ' % nn_min, flabel = ('%.2f $yr^{-1}$' % fobs_yrs[ff]),
            nshow=nshow, civals = [50,98,], color='tab:blue')
    draw_reals(ax, xx, y2[ff], rlabel='med (sample %d), ' % nn_med, flabel = ('%.2f $yr^{-1}$' % fobs_yrs[ff]),
            nshow=nshow, civals = [50,98,], color='tab:orange')
    draw_reals(ax, xx, y3[ff], rlabel='max (sample %d), ' % nn_max, flabel = ('%.2f $yr^{-1}$' % fobs_yrs[ff]),
            nshow=nshow, civals = [50,98,], color='tab:green')
    draw_reals(ax, xx, y4[ff], rlabel='max (sample %d), ' % nn_max, flabel = ('%.2f $yr^{-1}$' % fobs_yrs[ff]),
            nshow=nshow, civals = [50,98,], color='tab:grey')
    ax.legend()
    ax.set_title(title)
    return fig

title = ('50%% and 98%% confidence intervals of %d realizations, lmax=%d\nSelected by mean $C_l/C_0$ over all freqs'
                % (nreals, lmax))
fig = ClC0_min_max_med(lvals, Cl_best, bb_min, bb_med, bb_max,title=title, ff=fidx)


## Plot Clg0 for min, max, med

In [ ]:
def Clg0_min_max_med(lvals, Cl_best, bb_min, bb_med, bb_max, title='', ff=fidx):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$f$ [yrs$^{-1}$]', ylabel='$C_{\ell>0}/C_0$')

    xx = fobs_yrs # (l)
    y1 = Cl_best[bb_min] # (F,R,l)
    y2 = Cl_best[bb_med] # (F,R,l)
    y3 = Cl_best[bb_max] # (F,R,l)
    y4 = Cl_best[nsort[0]] # (F,R,l)

    y1 = np.sum(y1[:,:,1:], axis=-1)/y1[:,:,0] # (F,R,l)
    y2 = np.sum(y2[:,:,1:], axis=-1)/y2[:,:,0] 
    y3 = np.sum(y3[:,:,1:], axis=-1)/y3[:,:,0] 
    y4 = np.sum(y4[:,:,1:], axis=-1)/y4[:,:,0] 
    
    y1=np.swapaxes(y1,0,1)
    y2=np.swapaxes(y2,0,1)
    y3=np.swapaxes(y3,0,1)
    y4=np.swapaxes(y4,0,1)

    draw_reals(ax, xx, y1, rlabel='min (sample %d), ' % nn_min, 
            nshow=nshow, civals = [50,98,], color='tab:blue')
    draw_reals(ax, xx, y2, rlabel='med (sample %d), ' % nn_med, 
            nshow=nshow, civals = [50,98,], color='tab:orange')
    draw_reals(ax, xx, y3, rlabel='max (sample %d), ' % nn_max, 
            nshow=nshow, civals = [50,98,], color='tab:green')
    draw_reals(ax, xx, y4, rlabel='max (sample %d), ' % nsort[0], 
            nshow=nshow, civals = [50,98,], color='tab:gray')
    ax.legend()
    ax.set_title(title)
    return fig

title = ('50%% and 98%% confidence intervals of %d realizations, lmax=%d\nSelected by mean $C_l/C_0$ over all freqs'
                % (nreals, lmax))
fig = Clg0_min_max_med(lvals, Cl_best, bb_min, bb_med, bb_max,title=title, ff=fidx)




## Plot Clvf for med only

In [ ]:
def Clvf_med(lvals, Cl_best, bb_min, bb_med, bb_max, title='', ff=fidx):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$f$ [yrs$^{-1}$]', ylabel='$C_{\ell}$')

    xx = fobs_yrs # (F)
    yy = Cl_best[bb_med] # (F,R,l)
    yy = np.swapaxes(yy, 0, 2) # (l,R,F)


    colors=np.array(['','#0a5da4', '#00ba44', '#ff9503',
                     '#ff471a', '#845b98', '#474747'])

    for ll in range(1,7):
        draw_reals(ax, xx, yy[ll], rlabel=('$\ell=%d$' % ll),
                   nshow=nshow, civals=[50,], color=colors[ll])

    ax.legend(ncols=3)
    ax.set_title(title)
    return fig

title = ('Sample %d (Median of mean($C_\ell/C_0$))\n50%% confidence intervals of %d realizations'
                % (nn_med, nreals))
fig = Clvf_med(lvals, Cl_best, bb_min, bb_med, bb_max,title=title, ff=fidx)




In [ ]:
def Clvf_1real(lvals, Cl_best, bb_min, bb_med, bb_max, ff=fidx, rr=0):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$f$ [yrs$^{-1}$]', ylabel='$C_\ell$')

    xx = fobs_yrs # (F)
    yy = Cl_best[bb_med] # (F,R,l)
    yy = np.swapaxes(yy, 0, 2) # (l,R,F)


    colors=np.array(['','#0a5da4', '#00ba44', '#ff9503',
                     '#ff471a', '#845b98', '#474747'])
    for ll in range(1,7):
        ax.plot(xx, yy[ll,rr], label=('$\ell=%d$' % ll), color=colors[ll], marker='o', linestyle='-')

    ax.legend(ncols=3)
    title = ('Sample %d (Median of mean($C_\ell/C_0$))\n%dth Realization'
                % (nn_med, rr))
    ax.set_title(title)
    return fig

for rr in range(0,1):
    fig = Clvf_1real(lvals, Cl_best, bb_min, bb_med, bb_max, ff=fidx, rr=rr)




In [ ]:
def ClC0vf_med(lvals, Cl_best, bb_min, bb_med, bb_max, title='', ff=fidx):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$f$ [yrs$^{-1}$]', ylabel='$C_{\ell}/C_0$')

    xx = fobs_yrs # (F)
    yy = Cl_best[bb_med,:,:,1:]/Cl_best[bb_med,:,:,0,np.newaxis] # (F,R,l)
    yy = np.swapaxes(yy, 0, 2) # (l,R,F)


    colors=np.array(['','#0a5da4', '#00ba44', '#ff9503',
                     '#ff471a', '#845b98', '#474747'])

    for ll in range(1,7):
        draw_reals(ax, xx, yy[ll], rlabel=('$\ell=%d$' % ll),
                   nshow=nshow, civals=[50,], color=colors[ll])

    ax.legend(ncols=3)
    ax.set_title(title)
    return fig

title = ('Sample %d (Median of mean($C_\ell/C_0$))\n50%% confidence intervals of %d realizations'
                % (nn_med, nreals))
fig = ClC0vf_med(lvals, Cl_best, bb_min, bb_med, bb_max,title=title, ff=fidx)




In [ ]:
def ClC0vf_1real(lvals, Cl_best, bb_min, bb_med, bb_max, ff=fidx, rr=0):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$f$ [yrs$^{-1}$]', ylabel='$C_\ell/C_0$')

    xx = fobs_yrs # (F)
    yy = Cl_best[bb_med] # (F,R,l)
    yy = yy[:,:,1:]/yy[:,:,0,np.newaxis]
    yy = np.swapaxes(yy, 0, 2) # (l,R,F)


    colors=np.array(['','#0a5da4', '#00ba44', '#ff9503',
                     '#ff471a', '#845b98', '#474747'])
    for ll in range(1,7):
        ax.plot(xx, yy[ll,rr], label=('$\ell=%d$' % ll), color=colors[ll], marker='o', linestyle='-')

    ax.legend(ncols=3)
    title = ('Sample %d (Median of mean($C_\ell/C_0$))\n%dth Realization'
                % (nn_med, rr))
    ax.set_title(title)
    return fig

for rr in range(0,1):
    fig = ClC0vf_1real(lvals, Cl_best, bb_min, bb_med, bb_max, ff=fidx, rr=rr)




# Compare to Nihan's Plot

In [ ]:
def ClC0_double(lvals, Cl_best, bb_min, bb_med, bb_max, ff=fidx, rr=0):
    fig, (ax1,ax2) = plot.figax(figsize=(10,4), ncols=2, nrows=1, 
                                sharey=True, xscale='linear', yscale='linear')

    x1 = lvals[1:7]
    x2 = fobs[0:7] # (F)
    yy = Cl_best[bb_med,0:7,:,:] # (F,R,l)
    y1 = yy[:,:,1:7]/yy[:,:,0,np.newaxis] # (F,R,l)
    y2 = yy[:,:,0:7]/yy[:,:,0,np.newaxis] # (F,R,l)
    y2 = np.swapaxes(y2, 0, 2) # (l,R,F)

    print(x1.shape, y1.shape, x2.shape, y2.shape)

    colors=np.array(['brown','#0a5da4', '#00ba44', '#ff9503',
                     '#ff471a', '#845b98', '#474747'])
    for ff in range(0,7):
        ax1.plot(x1, y1[ff,rr], label=('$f=%.2f$ nHz' % fobs_nHz[ff]), color=colors[ff], marker='o', linestyle='-')
    for ll in range(1,7):
        ax2.plot(x2, y2[ll,rr], label=('$\ell=%d$' % ll), color=colors[ll], marker='o', linestyle='-')

    ax1.legend(loc='upper right', fontsize=8)
    ax1.set_xlabel('$\ell$')
    ax1.set_ylabel('$C_\ell/C_0$')
    ax1.set_ylim(0,0.15)


    ax2.legend(ncols=3, loc='upper right', fontsize=8)
    ax2.set_xlabel('f [Hz]')
    ax2.set_xlim(0.2e-8, 1.0e-8)

    title = ('Sample %d (Median of mean($C_\ell/C_0$))\n%dth Realization'
                % (nn_med, rr))
    fig.suptitle(title)
    fig.tight_layout()
    return fig

for rr in range(0,10):
    fig = ClC0_double(lvals, Cl_best, bb_min, bb_med, bb_max, ff=fidx, rr=rr)




In [ ]:
print(fobs*10**8)
ff_trunc = fobs[0:7]
print(ff_trunc*10**8)

### Select min, med, max just for this frequency bin

In [ ]:
bb_max = np.argmax(ClC0_mean)
bb_med = argmedian(ClC0_mean, side='left')
nn_min, nn_max, nn_med = nsort[bb_min], nsort[bb_max], nsort[bb_med]
print('Min: Sample %d, Median: Sample %d, Max: Sample %d' % (nn_min, nn_med, nn_max))

In [ ]:
# nside = NSIDE
# npix = hp.nside2npix(nside)
# lmax6 = 6
# lvals6 = np.arange(lmax6+1)
# nbest6 = 10
# Cl_best6 = np.zeros((nbest, nfreqs, nreals, lmax6+1 ))
# moll_hc_best6 = np.zeros((nbest6, nfreqs, nreals, npix))
# for nn in range(nbest):
#     print('on nn=%d out of nbest=%d' % (nn,nbest))
#     moll_hc_best6[nn,...], Cl_best6[nn,...] = sph_harm_from_hc(hc_ss[nsort[nn]], 
#                                             hc_bg[nsort[nn]],
#                                             nside=nside, lmax=lmax6)

<!-- ### Plot Results -->

In [ ]:
# fig1 = plot_Cl_medians(lvals6, Cl_best6, lmax6)
# fig2 = plot_ClC0_medians(lvals6, Cl_best6, lmax6)
# fig3 = plot_Clg0_medians(Cl_best6, lmax6)

In [ ]:
# fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')

# xx = fobs_yrs
# yy = np.sum(Cl_best6[:,:,:,1:], axis=-1)/Cl_best6[:,:,:,0,] # (B,F,R)

# colors = cm.rainbow(np.linspace(1, 0, 5))
# bb=0
#     # label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
# ax.plot(xx, np.median(yy[bb,:,:], axis=1), color='k')
# for pp in [50, 98]:
#     percs = pp/2
#     percs = [50-percs, 50+percs]
#     ax.fill_between(xx, *np.percentile(yy[bb,:,:], percs, axis=1), alpha=0.15, color='k')

# for rr in range(20):
#     ax.plot(xx, yy[bb,:,rr], linestyle=':', alpha=0.75)
# ax.legend()
# ax.set_title('50%% and 98%% confidence intervals from %d realizations of the %dth samples\nlmax=%d' % (nreals,nsort[bb], lmax6))

# Using 12.5 yr data

In [ ]:
# amp_ref12 = 1.53*10**-15
# nsort12, fidx12, hc_tt12 = rank_samples(hc_ss, hc_bg, fobs, dfobs, amp_ref=amp_ref12, ret_all=True)
# nbest = 25
# hc_ref12 = amp_ref12 * np.sqrt(fobs[fidx]/dfobs[fidx])
# print(hc_ref12)


In [ ]:
# # amplitudes from sorting
# fig, ax = plot.figax(figsize=(7,5), xlabel='Sample', ylabel='$h_{c,\mathrm{tot}}(f=1/\mathrm{yr})$',
#                      xscale='linear')
# ax.scatter(nsort12, hc_tt[nsort12], s=20, marker='o', c=np.arange(len(nsort)), 
#            cmap='jet_r')
# ax.axhline(hc_ref12, label='12.5 yr ref: %.2e' % (hc_ref12), linestyle='dashed', color='tab:red', alpha=0.5)
# # plt.scatter(np.arange(nsamps), hc_tt[nsort], s=3, label= r'15 yr ref: $A=$' + '%.2f' % amp_ref15)

# plt.yscale('log')
# plt.xlabel('Samples')
# plt.ylabel('$h_c(f=1\mathrm{yr}^{-1})$')
# plt.legend()

In [ ]:
# print(nsort12[:5])
# for nn in nsort12[:5]:
#     print('nn=%d' % nn)
#     fig = plot.plot_bg_ss(fobs, hc_bg[nn], hc_ss[nn,:,:,:100]) # just 100 loudest, cant see more anyway
#     ax = fig.axes[0]
#     ax.axhline(hc_ref12, label='12.5 yr ref: %.2e' % (hc_ref12),
#                 linestyle='dashed', color='tab:red', alpha=0.5)

#     ax.axvline(1/10, color='k', alpha=0.5, linestyle='dashed', label='$f=1/10\mathrm{yr}$')
#     ax.set_title('Sample %d' % nn)
#     ax.legend()

In [ ]:
print(nfreqs)

# Maps for funsies

In [ ]:
rr=0
moll_hc = moll_hc_best[0]
F=nfreqs
L=nloudest

fig, axs = plt.subplots(nrows=10, ncols=int(F/10))
for ff in range(F):
    title='$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
    row, col = int(ff/3), ff%3
    # print(row,col)
    plt.axes(axs[row,col])
    hp.mollview(moll_hc[ff,rr], title=title, hold=True)

fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
fig.set_size_inches(18, 30)
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

In [ ]:
Cl = Cl_best[0]

moll_sh = map_from_sph_harm(Cl)

rr=0
fig, axs = plt.subplots(nrows=10, ncols=int(F/10))
for ff in range(F):
    title='$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
    row, col = int(ff/3), ff%3
    # print(row,col)
    plt.axes(axs[row,col])
    hp.mollview(moll_sh[ff,rr], title=title, hold=True)

fig.suptitle('%d Loudest Single Sources, Recovered from Spherical Harmonics' % L, y=.895)
fig.tight_layout()
fig.set_size_inches(18, 30)
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

In [ ]:
ff=fidx
title = '$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, nfreqs)
hp.mollview(moll_hc[ff,rr], title='Randomly Placed: '+title, hold=False)
hp.mollview(moll_sh[ff,rr], title='Reconstructed from Harmonics: '+title, hold=False)


In [ ]:
print(npix)

In [ ]:
# Cl = Cl_best6[0]
# moll_sh6 = map_from_sph_harm(Cl)

# ff=fidx
# title = '$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
# hp.mollview(moll_hc[ff,rr], title='Randomly Placed: '+title, hold=False)
# hp.mollview(moll_sh6[ff,rr], title='Reconstructed from Harmonics: '+title, hold=False)


In [ ]:
# for lmaxb in (4,5,6,7,8,9,10):
#     print(lmaxb)
#     moll_hcb, Clb = sph_harm_from_hc(hc_ss[nsort[0]], hc_bg[nsort[0]], 
#                                     nside=nside, lmax=lmaxb)
#     moll_shb = map_from_sph_harm(Clb)
#     hp.mollview(moll_shb[ff,rr], title=(('Reconstructed from %d Harmonics: ' % lmaxb) + title), hold=False)

In [ ]:
# for lmaxb in (1,2,3,15,20):
#     print(lmaxb)
#     moll_hcb, Clb = sph_harm_from_hc(hc_ss[nsort[0]], hc_bg[nsort[0]], 
#                                     nside=nside, lmax=lmaxb)
#     moll_shb = map_from_sph_harm(Clb)
#     hp.mollview(moll_shb[ff,rr], title=(('Reconstructed from %d Harmonics: ' % lmaxb) + title), hold=False)

In [ ]:
# for lmaxb in (30, 50):
#     print(lmaxb)
#     moll_hcb, Clb = sph_harm_from_hc(hc_ss[nsort[0]], hc_bg[nsort[0]], 
#                                     nside=nside, lmax=lmaxb)
#     moll_shb = map_from_sph_harm(Clb)
#     hp.mollview(moll_shb[ff,rr], title=(('Reconstructed from %d Harmonics: ' % lmaxb) + title), hold=False)

## Compare lmax for harmonics

In [ ]:
lmaxes=(0,1,2,3,4,5,6,7,8,9,10,15,20,25,30,50)
moll_maxes = np.zeros((len(lmaxes), nfreqs, nreals, npix))
for ll in range(len(lmaxes)):
    print(lmaxes[ll])
    moll_hcb, Clb = sph_harm_from_hc(hc_ss[nsort[0]], hc_bg[nsort[0]], 
                                    nside=nside, lmax=lmaxes[ll])
    moll_maxes[ll] = map_from_sph_harm(Clb)

In [ ]:
print(sspath)

In [ ]:


rr=0
ff=fidx
fig, axs = plt.subplots(nrows=4, ncols=4)
for ll in range(len(lmaxes)):
    title='lmax= %d, f=%.2f' % (lmaxes[ll], fobs_yrs[ff])
    row, col = int(ll/4), ll%4
    # print(row,col)
    plt.axes(axs[row,col])
    hp.mollview(moll_maxes[ll,ff,rr], title=title, hold=True)

fig.suptitle('%d Loudest Single Sources, Recovered from Spherical Harmonics' % L, y=.895)
fig.tight_layout()
fig.set_size_inches(18, 12)
fig.savefig(sspath+'maps_from_harmonics.png', dpi=300)
# fig